In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         pass
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf
import cudf
import cuml
import cupy
import matplotlib.pyplot as plt

In [3]:
data_path = '../input/h-and-m-personalized-fashion-recommendations/'
image_path = '../input/h-and-m-personalized-fashion-recommendations/images/'

In [4]:
df = pd.read_csv(data_path+'transactions_train.csv', dtype={'article_id': str, 't_dat': object, 'customer_id': str, 'price': float, 'sales_channel_id': int})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   t_dat             object 
 1   customer_id       object 
 2   article_id        object 
 3   price             float64
 4   sales_channel_id  int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 1.2+ GB


 - Pre-processing

In [5]:
df['t_dat'] = pd.to_datetime(df['t_dat'])
df['timestamp'] = pd.to_datetime(df['t_dat'], unit='ms').astype(np.int64)//1e9
df['customer_id'] = df['customer_id'].astype('str')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  


In [6]:
df.sample(5)

,t_dat,customer_id,article_id,price,sales_channel_id,timestamp
14650937,2019-07-27,e9718eb753472423db54fd575188c252caea6c7ffd5f2e...,0598515006,0.016932,2,1.564186e+09
450612,2018-09-29,4053689f05dc39022adf1bf89e37212e47d9eda610db97...,0672598001,0.017610,1,1.538179e+09
24855061,2020-04-20,8a798c28c48eb3bb1c71ed93f9f28962eb9e7cdf97b4b4...,0226959008,0.015797,2,1.587341e+09
22884844,2020-02-29,f5a6793b576cf2d0ab60b5ee6d0e062d69b9b57fb48a51...,0776237011,0.025407,1,1.582934e+09
14173259,2019-07-20,c7e635c0bc89ae7ea4fa5cd7abbe67d3fc3966bd3abb24...,0695581001,0.015237,1,1.563581e+09


In [7]:
null_cnt = df.isna().sum()
null_cnt

t_dat               0
customer_id         0
article_id          0
price               0
sales_channel_id    0
timestamp           0
dtype: int64

In [8]:
#Unique items in each field
total_transactions = len(df)
for col in ['customer_id', 'article_id']:
    uq = len(df[col].unique())
    print('Unique in', col, uq, 'Density:', total_transactions/uq)

Unique in customer_id 1362281 Density: 23.334630667241193
Unique in article_id 104547 Density: 304.057734798703


In [9]:
uniq_customer = list(df['customer_id'].unique())
uniq_article = list(df['article_id'].unique())
print(len(uniq_customer), len(uniq_article))

1362281 104547


In [10]:
uniq_customer[:3]

['000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318',
 '00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2',
 '00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4c73235dccbbc132280']

In [11]:
sub_df = df.sample(90_000, random_state=26)
sub_df

,t_dat,customer_id,article_id,price,sales_channel_id,timestamp
17258814,2019-09-28,ed09247cfe0c4f3ca9620169fb0a9562910a6dfe064521...,0757303002,0.024390,2,1.569629e+09
18385881,2019-10-28,b8a2153e7c93981cd110b6c8ed93236e5b0d27ecc3cc98...,0781833003,0.025407,2,1.572221e+09
10647175,2019-05-23,0e9d42f81879d01a3b2d8d4725585e3b77ba8208405d77...,0708845001,0.013542,1,1.558570e+09
20519357,2019-12-23,2a396024b9f9c8d41d0d7b3b70c63ca80d1c674bcf0e29...,0808276001,0.030492,1,1.577059e+09
29807461,2020-08-02,55f6ab9a67b10abc185893ef274ad49405beb1e1310783...,0874669001,0.025407,2,1.596326e+09
...,...,...,...,...,...,...
17648977,2019-10-07,83037a975e7f214f20a00afb2c6348a1f14fff6816a953...,0293510002,0.013983,2,1.570406e+09
6074102,2019-02-11,3d76c3aa6a970f0a483349a256bdc23f91b2cbcb1d3fe1...,0649662002,0.067780,2,1.549843e+09
13299980,2019-07-04,81f915bab7af80b4ac89256e71a7fbe5f16fad404cc359...,0731425002,0.027102,1,1.562198e+09
14347612,2019-07-23,e26fc9272a09c42f197e7e36b69b86b8b0b6ed54ab020f...,0803522002,0.050831,2,1.563840e+09


In [12]:
itemSeq = sub_df.groupby('customer_id')['article_id'].apply(lambda x: x.to_string(index=False, header=False))
itemSeq

customer_id
00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2    0678079005\n0657291004
00009d946eec3ea54add5ba56d5210ea898def4b46c68570cf0096d962cacc75                0669713001
0000b2f1829e23b24feec422ef13df3ccedaedc85368e6664d04ca30b2f8daff    0589544003\n0714828001
0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37e011580a479e80aa94                0710953002
00045b41ba4df7a2e6c09093ec92873ea43a137cf2b24cff1a387f75a24782a9                0790791005
                                                                             ...          
fffae8eb3a282d8c43c77dd2ca0621703b71e90904dfde2189bdd644f59071dd                0158340001
fffbe76d05cba1b9bdf14bf67e58e7b73d4c2e3418f13cef418a01a6d37946e5                0739575004
fffdc8b7c7c181c658b354b2b408bc690dfaeb80ebe56aca9d0ce632df9a6660                0628979001
ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e4747568cac33e8c541831                0717884001
ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1778d0116cffd259264              

In [13]:
orderedItems = []
customers = []
items = []
for key, seq in itemSeq.iteritems():
    s = seq.split('\n')
#     print(key)
    customers.append(key)
    items.append(' '.join(s))
    if len(s) > 1:
        orderedItems.append(' '.join(s))
orderedItems

['0678079005 0657291004',
 '0589544003 0714828001',
 '0744893003 0578476001 0570002008',
 '0669215002 0759796002',
 '0800800001 0767133001',
 '0576661008 0730012005',
 '0896559001 0861477001',
 '0717490003 0448509018',
 '0160442007 0570319001',
 '0714826002 0677736003',
 '0759871002 0829180001',
 '0743642001 0570189003',
 '0682054001 0803378001',
 '0797343001 0888010003',
 '0733852001 0844090002',
 '0759326008 0812032001',
 '0877261003 0730683001 0705021001',
 '0456163065 0457466002',
 '0817491003 0744073001',
 '0695072004 0843793004',
 '0780209001 0757904001',
 '0698302007 0724791006',
 '0764838003 0693196002',
 '0603584018 0706890001 0647434001',
 '0636323003 0636323001',
 '0690936004 0842605004',
 '0688771001 0756105001',
 '0676512002 0679278007',
 '0717885001 0624486005',
 '0747196001 0758942001',
 '0843373003 0733045002',
 '0749699001 0799365011 0516000044 0736573002',
 '0542464001 0693917002 0681376001',
 '0891591003 0624645006',
 '0876740002 0642189001',
 '0620425015 0697564018'

In [14]:
final_df = pd.DataFrame({'customer_id':customers, 'order_seq': items})
final_df

,customer_id,order_seq
0,00007d2de826758b65a93dd24ce629ed66842531df6699...,0678079005 0657291004
1,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,0669713001
2,0000b2f1829e23b24feec422ef13df3ccedaedc85368e6...,0589544003 0714828001
3,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0710953002
4,00045b41ba4df7a2e6c09093ec92873ea43a137cf2b24c...,0790791005
...,...,...
80525,fffae8eb3a282d8c43c77dd2ca0621703b71e90904dfde...,0158340001
80526,fffbe76d05cba1b9bdf14bf67e58e7b73d4c2e3418f13c...,0739575004
80527,fffdc8b7c7c181c658b354b2b408bc690dfaeb80ebe56a...,0628979001
80528,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,0717884001


In [15]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(orderedItems)

In [16]:
total_items = len(tokenizer.word_index) + 1
total_items

12317

In [17]:
input_sequences = []
for line in orderedItems:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [18]:
max_seq_len = max([len(inp) for inp in input_sequences])
max_seq_len

8

In [19]:
seq = tf.keras.preprocessing.sequence.pad_sequences(input_sequences, maxlen=max_seq_len-1, padding='pre')

In [20]:
seq[0]

array([   0,    0,    0,    0,    0, 3140, 3141], dtype=int32)

In [21]:
X = seq[:,:-1]
labels = seq[:,-1]
# One-hot encode with keras convert list to a categorical. The number of classes which is my number of words.
Y = tf.keras.utils.to_categorical(labels, num_classes=total_items)


In [22]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(total_items, 100, input_length=max_seq_len-1))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150)))
model.add(tf.keras.layers.Dense(total_items, activation='softmax'))
adam = tf.keras.optimizers.Adam()
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
history = model.fit(X, Y, epochs=25, verbose=1)
print(model)

2022-03-16 11:56:24.858748: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-16 11:56:24.863410: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-16 11:56:24.864129: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-16 11:56:24.865183: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Epoch 1/25


2022-03-16 11:56:34.892961: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


296/296 [==============================] - 7s 9ms/step - loss: 9.4141 - accuracy: 2.1119e-04
Epoch 2/25
296/296 [==============================] - 3s 9ms/step - loss: 9.1972 - accuracy: 6.3358e-04
Epoch 3/25
296/296 [==============================] - 3s 9ms/step - loss: 8.7225 - accuracy: 0.0013
Epoch 4/25
296/296 [==============================] - 3s 9ms/step - loss: 8.0945 - accuracy: 0.0019
Epoch 5/25
296/296 [==============================] - 3s 9ms/step - loss: 7.2282 - accuracy: 0.0071
Epoch 6/25
296/296 [==============================] - 3s 10ms/step - loss: 6.3210 - accuracy: 0.0207
Epoch 7/25
296/296 [==============================] - 3s 9ms/step - loss: 5.4522 - accuracy: 0.0629
Epoch 8/25
296/296 [==============================] - 3s 9ms/step - loss: 4.6180 - accuracy: 0.1335
Epoch 9/25
296/296 [==============================] - 3s 9ms/step - loss: 3.8431 - accuracy: 0.2250
Epoch 10/25
296/296 [==============================] - 3s 9ms/step - loss: 3.1029 - accuracy: 0.3360
E

In [23]:
def findNext12(seed_text):
    next_items = 12
    #Prediction
    print(seed_text)
    ans = ""
    for _ in range(next_items):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = tf.keras.preprocessing.sequence.pad_sequences([token_list], maxlen=max_seq_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list, verbose=0)[0])
#         print(predicted)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
        ans += " " + output_word
    return ans[1:]

In [24]:
findNext12('0589544003 0714828001')

0589544003 0714828001


'0539723001 0745155001 0695457004 0786304002 0770315007 0700935001 0464679008 0505882002 0904416001 0687524012 0707983004 0682550001'

In [25]:
final_df['prediction'] = final_df['order_seq'][:50].apply(lambda x: findNext12(x))
final_df

0678079005 0657291004
0669713001
0589544003 0714828001
0710953002
0790791005
0253448001
0720228001
0742116001
0657165002
0567534007
0880774001
0579541072
0698273002
0632813002
0813094005
0723380003
0673396002
0624048001
0568597023
0759392008
0695632002
0628326002
0773015001
0711649001
0469137017
0725662002
0909081004
0762846001
0785710001
0707470001
0539723001
0701853006
0817491001
0844059003
0733979001
0744893003 0578476001 0570002008
0852643001
0772176001
0873829001
0811835004
0834368002
0705988003
0633785009
0821115007
0669215002 0759796002
0816423002
0760084006
0681175003
0811835003
0153115019


,customer_id,order_seq,prediction
0,00007d2de826758b65a93dd24ce629ed66842531df6699...,0678079005 0657291004,0669091028 0811099001 0679853020 0779324004 07...
1,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,0669713001,0713428004 0684209013 0809549001 0812189001 06...
2,0000b2f1829e23b24feec422ef13df3ccedaedc85368e6...,0589544003 0714828001,0539723001 0745155001 0695457004 0786304002 07...
3,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0710953002,0718381002 0713428004 0684209013 0809549001 07...
4,00045b41ba4df7a2e6c09093ec92873ea43a137cf2b24c...,0790791005,0718381002 0713428004 0684209013 0809549001 07...
...,...,...,...
80525,fffae8eb3a282d8c43c77dd2ca0621703b71e90904dfde...,0158340001,NaN
80526,fffbe76d05cba1b9bdf14bf67e58e7b73d4c2e3418f13c...,0739575004,NaN
80527,fffdc8b7c7c181c658b354b2b408bc690dfaeb80ebe56a...,0628979001,NaN
80528,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,0717884001,NaN
